In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime as dt

#sending email
import smtplib

In [2]:
URL = "https://www.amazon.com.au/s?k=books&crid=1EISML3B83NV&sprefix=book%2Caps%2C242&ref=nb_sb_noss_1"
HEADERS = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.1 Safari/605.1.15",
                }
response = requests.get(URL,headers = HEADERS)
soup = BeautifulSoup(response.content,"html.parser")
pretty_soup = BeautifulSoup(soup.prettify(),"html.parser")

links = pretty_soup.find_all("a",attrs={'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})
list_of_books_links =[]
for book_link in links:
    list_of_books_links.append(book_link.get('href'))


data_frame = {"Title":[],"Author":[],"Rating":[], "Price":[]}
for link in list_of_books_links:
    #new_book = requests.get("https://www.amazon.com.au/"+link,headers= HEADERS)
    new_link = "https://www.amazon.com.au/"+link
    print(new_link)
    

https://www.amazon.com.au//sspa/click?ie=UTF8&spc=MTo4ODY2NDkwNTU3ODQxNjIyOjE3MTU2MDcxMTM6c3BfYXRmOjMwMDAxMjk2NDEzMDU2Mjo6MDo6&url=%2FSpelling-Life-Uncovering-simplicity-spelling%2Fdp%2F0367645696%2Fref%3Dsr_1_2_sspa%3Fcrid%3D1EISML3B83NV%26dib%3DeyJ2IjoiMSJ9.YbhDoTV1pens9T9dC6H86G83bsN18cY-gjukl90rHVTLSLb4g9DlwIJlXebOaxeiBk95A_SSEKKDPvd_hanOie1LwgzI5YY3IpZ-13kaHGkzE3J307avBu0BttmcWVyZTOH9xZmwLpYYGq5tMqVzjvod3ftohEc_ThgVTRVsOKZisAWcdx6sZ5eTjI3So1DJY6pq3wKFTm0L_RB1bECkHFb6gees1Ov4g2DBkKnbRpgKcky88l1ccSegeexCgdVArHTjMwqZM6u7XQtTiXqUNE7OCcOFamAPD20d6qGWInw.1ZJ14s01BXYPOvQP0nXF7r1Zb7yBbIdXbn9KIBbAAG8%26dib_tag%3Dse%26keywords%3Dbooks%26qid%3D1715607113%26sprefix%3Dbook%252Caps%252C242%26sr%3D8-2-spons%26sp_csd%3Dd2lkZ2V0TmFtZT1zcF9hdGY%26psc%3D1
https://www.amazon.com.au//sspa/click?ie=UTF8&spc=MTo4ODY2NDkwNTU3ODQxNjIyOjE3MTU2MDcxMTM6c3BfYXRmOjMwMDAxMDI3MzgwNjg2Mjo6MDo6&url=%2FArt-Game-Design-Lenses-Third%2Fdp%2F1138632058%2Fref%3Dsr_1_3_sspa%3Fcrid%3D1EISML3B83NV%26dib%3DeyJ2IjoiMSJ9.YbhD